# 데이터 수집 프로젝트

* 팀 주제 : 대형 서점 4곳 월별 베스트 셀러 20위까지 데이터 수집
* 대형 서점 선정 -> 교보문고, 영풍문고, 예스24, 알라딘
* 각자 한개의 사이트를 맡아서 22년 7월 ~ 23년 6월까지 1년치의 베스트셀러 데이터를 수집


* 맡은 사이트 : 예스24 ('https://www.yes24.com')
* 예스24의 22년 7월부터 23년 6월까지 월별 베스트셀러 20위까지 데이터 수집 항목에 맞춰서 크롤링

### 6월 28일 (Day 3)

* 최종 완성된 코드 

* 수집 항목 : ISBN, 책 제목(title), 책 장르(category), 저자(writer), 평점(grade/point), 판매가격(price), 발행일(date)
* 년도와 월을 파라미터로 넣으면 돌아가는 함수 작성 

1 root_url에서 각 year와 month를 받아 원하는 년도+월에 월간 베스트 url을 세팅

2 월간 베스트 목록에서 책 제목 요소를 전부 받아온 후 해당 제목의 href 속성을 가져옴(총 20개)

3 가져온 href 속성은 for문을 돌려서 한 페이지씩 돌릴 수 있도록 book_url을 설정

4 설정된 book_url은 각 책의 상세페이지이므로 상세페이지에서 설정한 항목을 수집

5 수집된 항목은 DataFrame에 담아서 csv로 저장


In [2]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests, time, os

def bestSeller(year, month) :
    root_url = 'https://www.yes24.com'
    
    add_year = 'https://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=09&year='
    add_month = '&month='
    
    set_url = add_year + year + add_month + month
    # 월간 베스트 url에서 연도(year)와 달(month)가 바뀌도록 url 세팅
    
    book_list = []
    
    url = requests.post(set_url)
    bs = BeautifulSoup(url.text, 'html.parser')
    books = bs.select('td.goodsTxtInfo > p:nth-child(1) > a:nth-child(1)')
    # 첫 화면에서 request로 html화면을 파싱해서 각 책들의 제목의 내용을 저장
    
    for book in books :
        
        # 책들의 제목 요소 중에서 href 링크만 따와서 root_url과 합쳐서 각 책들의 상세페이지 url 생성
        book_url = root_url + book.attrs['href']
        book_data = requests.post(book_url)
        book_bs = BeautifulSoup(book_data.text, 'html.parser')
        
        # 각 항목(column)별 데이터 뽑아내기
        # isbn, title, category, writer, point, price, date (isbn, 제목, 장르, 저자, 평점, 가격, 발행일)
        select_isbn = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > th'
        if book_bs.select(select_isbn)[0].text != 'ISBN13' :
            isbn13 = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(4) > td'
        else :
            isbn13 = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td' 
        isbn = book_bs.select(isbn13)[0].text    

        select_title = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
        title = book_bs.select(select_title)[0].text 

        select_category = '#infoset_goodsCate > div.infoSetCont_wrap > dl:nth-child(1) > dd > ul > li:nth-child(1) > a:nth-child(4)'
        category = book_bs.select(select_category)[0].text

        select_writer = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a'
        writer = book_bs.select(select_writer)[0].text

        select_point = '#spanGdRating > a > em'
        point = book_bs.select(select_point)[0].text

        select_price = book_bs.find_all('span', {'class' : 'nor_price'})
        price = select_price[0].text

        select_date = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(1) > td'
        date = book_bs.select(select_date)[0].text
    
        # 각 항목들 한 리스트에 담아서 리스트로 저장
        book_list.append([isbn, title, category, writer, point, price, date])
    
    # 컬럼명 지정
    list = ['ISBN', '책 제목', '도서 장르', '저자', '평점', '판매가격', '발행일']
    
    # pandas에 DataFrame를 사용해서 표를 생성
    # index_df는 표에서 index를 설정해서 첫 행은 띄어지도록 표시하기 위한 DataFrame
    # book_list는 데이터
    index_df = pd.DataFrame(columns=list)
    index_df.loc[0] = ['', '', '', '', '', '', '']
    index_df.index = [str(year) + '년' + str(month) + '월']
    
    book_list = pd.DataFrame(np.array(book_list), columns=list)
    book_list.index = range(1, len(book_list)+1)
    book_list = pd.concat([index_df, book_list])

    return book_list

month_val = 6
for month in range(1, 13) :
    
    if month < 7 :
        year = 2022
        month = month_val + month
    
    else :
        year = 2023
        month = month - month_val
    
    book_list= bestSeller(str(year), str(month))

    if not os.path.exists('bestSeller.csv') :
        book_list.to_csv('bestSeller.csv', encoding='CP949', mode = 'w')
    else :
        book_list.to_csv('bestSeller.csv', encoding='CP949', mode = 'a', header=False)
    
    print('YEAR : ' , year)
    print('MONTH : ', month)
    print('저장완료')
    

,ISBN,책 제목,도서 장르,저자,평점,판매가격,발행일
2023년6월,,,,,,,
1,9791168473690,세이노의 가르침,자기계발,세이노(SayNo),9.1,"6,480원",2023년 03월 02일
2,9791168415232,흔한남매 13,어린이,흔한남매,9.8,"13,050원",2023년 04월 25일
3,9791188331888,사장학개론,경제 경영,김승호,9.6,"22,500원",2023년 04월 19일
4,9791190299770,모든 삶은 흐른다,인문,로랑스 드빌레르,9.5,"15,120원",2023년 04월 03일
5,9791198013026,김미경의 마흔 수업,자기계발,김미경,9.8,"16,200원",2023년 02월 15일
6,9791191891287,메리골드 마음 세탁소,소설/시/희곡,윤정은,9.4,"13,500원",2023년 03월 06일
7,9791165347161,나의 돈 많은 고등학교 친구,자기계발,송희구,9.6,"16,200원",2023년 04월 28일
8,9791169791977,스즈메의 문단속,소설/시/희곡,신카이 마코토,9.6,"11,700원",2023년 01월 13일
9,9791158773120,건강과 다이어트를 동시에 잡는 7대 3의 법칙 채소·과일식,건강 취미,조승우,9.5,"17,100원",2022년 10월 20일
